#### Монтировка диска
для доступа к файлам с датасетами (*'train.xml'* и *'test_etalon.xml'*)




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Чтение данных из файлов
- Парсинг xml файлов в `pandas.dataframe` с помощью библиотеки `xml.etree.ElementTree`

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

  
def xml_to_df(file_name):
    tree = ET.parse(file_name)
    root = tree.getroot() [1]
    all_records = []
    for i, child in enumerate(root):
        record = {}
        for sub_child in child:
            key = sub_child.attrib['name']
            value = sub_child.text
            if value == 'NULL':
                value = np.nan
            if key == 'id' or key == 'twitid':
                value = int(value)
            record[key] = value
        all_records.append(record)
    return pd.DataFrame(all_records).set_index('id')


train_file = "/content/drive/MyDrive/classf/train.xml"
test_file = "/content/drive/MyDrive/classf/test_etalon.xml"
df_train = xml_to_df(train_file)
df_test = xml_to_df(test_file)

display(df_train)

# with open(train_file, 'r', encoding='utf-8') as train:
#     df_train = pd.read_xml(train.read(),  xpath="//column")

,twitid,date,text,beeline,mts,megafon,tele2,rostelecom,komstar,skylink
id,,,,,,,,,,
1,492367588165680000,1406224555,"@mkomov Максим, Вашем письмо мы получили. Наши...",0,NaN,NaN,NaN,NaN,NaN,NaN
2,492369449912380000,1406224999,«Мегафон» стал владельцем 50% акций «Евросети»,NaN,NaN,0,NaN,NaN,NaN,NaN
3,492369715378290000,1406225062,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...,NaN,-1,NaN,NaN,NaN,NaN,NaN
4,492369842205650000,1406225092,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...,NaN,1,NaN,NaN,NaN,NaN,NaN
5,492371322060950000,1406225445,"@parfenov1960 потому что МТС достало, а пчел н...",NaN,-1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4996,497357408235420000,1407414221,Блогеры и журналисты Ставрополя публично проте...,NaN,NaN,0,NaN,NaN,NaN,NaN
4997,497358002065010000,1407414362,В Крыму полностью отключили инфраструктуру «МТ...,NaN,-1,0,NaN,NaN,NaN,NaN
4998,497358112610070000,1407414389,Кавказский #МегаФон предлагает новым корпорати...,NaN,NaN,1,NaN,NaN,NaN,NaN


#### `установка вспомогательных библиотек`

In [ ]:
!pip install pymorphy2

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Предобработка датасета
- Приведение некоторых типов `obj` к `int` 
- Добавление столбца `class` (1, -1, 0 - суммарный класс твита какой-либо компании)
- Фильтрование полученных таблиц
- Добавление столбца `data` (список токенов без стоп-слов - предобработанный текст твитов с помощью библиотек `nltk` и `pymorphy2`)

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from pymorphy2 import MorphAnalyzer


def extract_tokens(text):   
    stop_words = set(stopwords.words('russian')) | set('без был вам вас ваш вон вот все всю вся всё весь про раз сам сих так там тем тех том тот тою будто тоже пока пора это теми того тоже тому туда хоть хотя мимо для его ему еще ещё или ими как кем ком кто мне мог мож мои мой моя моё над нам нас наш нее ней нем нет нею неё них оба она они оно под пор при про раз сам сих так там тем тех том тот тою три тут уже чем что эта эти это эту'.split())
    rus_chars = set(chr(i) for i in range(ord('а'), ord('а') + 32)) | set('ё')
    marks = set(punctuation) | set('«»—…')

    words = word_tokenize(text.replace('#', ' '), 'russian')
    def del_mark(word):
        if word and word[0] in marks: word = word[1:]
        if word and word[-1] in marks: word = word[:-1]
        return word
    words = [del_mark(word) for word in words]
    words = [word for word in words if not word.lower() in stop_words
            and len(word) > 2 and not set(word.lower()) - rus_chars]
    
    tokens = [pm2.parse(word)[0].normal_form for word in words 
              if word not in stop_words]
    return tokens


def transform_df(df):
    pd.set_option('mode.chained_assignment', None)

    companies = df.columns[3:]
    for column in companies:
        df[column] = pd.to_numeric(df[column], errors='coerce') # if error of int(), then NaN
    
    df['class'] = df[companies].max(axis=1)
    df = df[~df['class'].isna()]
    df['class'] = df['class'].astype('int64')

    df['data'] = df['text'].apply(extract_tokens)

    return df


pm2 = MorphAnalyzer()
df_train = transform_df(df_train)
df_test = transform_df(df_test)

display(df_train)
print()
print(df_train["class"].value_counts())

,twitid,date,text,beeline,mts,megafon,tele2,rostelecom,komstar,skylink,class,data
id,,,,,,,,,,,,
1,492367588165680000,1406224555,"@mkomov Максим, Вашем письмо мы получили. Наши...",0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,"[максим, ваш, письмо, получить, наш, сотрудник..."
2,492369449912380000,1406224999,«Мегафон» стал владельцем 50% акций «Евросети»,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,"[мегафон, стать, владелец, акция, евросеть]"
3,492369715378290000,1406225062,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,-1,"[мтс, россия, прислать, житель, херсонщина, со..."
4,492369842205650000,1406225092,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,"[видео, реклама, смех, мтс, супер]"
5,492371322060950000,1406225445,"@parfenov1960 потому что МТС достало, а пчел н...",NaN,-1.0,NaN,NaN,NaN,NaN,NaN,-1,"[мтс, достать, пчела, ненавидеть, детство, мёд]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4996,497357408235420000,1407414221,Блогеры и журналисты Ставрополя публично проте...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,"[блогер, журналист, ставрополь, публично, прот..."
4997,497358002065010000,1407414362,В Крыму полностью отключили инфраструктуру «МТ...,NaN,-1.0,0.0,NaN,NaN,NaN,NaN,0,"[крым, полностью, отключить, инфраструктура, м..."
4998,497358112610070000,1407414389,Кавказский #МегаФон предлагает новым корпорати...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1,"[кавказский, мегафон, предлагать, новый, корпо..."



 0    2258
-1    1585
 1     956
Name: class, dtype: int64


#### Обучение модели-классификатора `KNeighborsClassifier` 
- с различными параметрами и разными способами векторизации слов
- оценивание каждой модели с целью выбора наилучшей

In [ ]:
def knn_cv_score(X_tr, y_train, X_te, y_test, parameters, score_functions, acc_function, knn_class):
    """Takes train data, counts score over grid of parameters (all possible parameters combinations) 

    Parameters:
    X_tr (2d np.array): train set
    y_train (1d np.array): train labels
    X_te (2d np.array): test set
    y_test (1d np.array): test labels
    parameters (dict): dict with keys from {n_neighbors, metrics, weights, normalizers}, values of type list,
                       parameters['normalizers'] contains tuples (normalizer, normalizer_name), see parameters
                       example in your jupyter notebook
    acc_function (callable): function with input (y_true, y_predict) which outputs score metric
    knn_class (obj): class of knn model to fit

    Returns:
    dict: key - tuple of (normalizer_name, n_neighbors, metric, weight), value - score
    """
    output = {}
    for normalizer, normalizer_name in parameters['normalizers']:
        if normalizer:
            normalizer.fit(X_tr)
            #normalizer.fit_transform(df_train.data).toarray()
            X_train = normalizer.transform(X_tr).toarray()
            X_test = normalizer.transform(X_te).toarray()

        for n_neighbors in parameters['n_neighbors']:
            for metric in parameters['metrics']:
                for weight in parameters['weights']:
                    key = (normalizer_name, n_neighbors, metric, weight)
                    
                    classifier = knn_class(n_neighbors, weights=weight, metric=metric)
                    classifier.fit(X_train, y_train)
                    y_pred = classifier.predict(X_test)

                    output[key] = [score(y_test, y_pred, average='macro', zero_division=1) 
                                    for score in score_functions]
                    output[key].append(acc_function(y_test, y_pred))
    return output

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


count_vec = CountVectorizer(analyzer=lambda x: x) # max_df=0.8, min_df=10, stop_words='russian'
tf_idf = TfidfVectorizer(analyzer=lambda x: x)

parameters = {
    'n_neighbors': [i for i in range(1, 8)],
    'metrics': ['euclidean', 'cosine'],
    'weights': ['uniform', 'distance'],
    'normalizers': [(count_vec, 'CountVectorizer'), (tf_idf, 'TfidfVectorizer')]
}

results = knn_cv_score(df_train['data'], df_train['class'], df_test['data'], df_test['class'], 
                       parameters, [precision_score, recall_score, f1_score], accuracy_score, 
                       KNeighborsClassifier)

print("Различные параметры классификатора KNN и оценочные метрики для каждой модели:\n")
print("(normalizer, n_neighbors, metric, weight): [precision, recall, f1_score, accuracy]\n")
print(*[f"{key}:" + " [" + ', '.join([f"{score:.4f}"for score in scores]) + "]"
          for key, scores in results.items()], sep='\n')

best_model = max(results.items(), key=lambda t: t[1][-1])
print("\nНаилучшая модель по аккуратности:\n", *best_model, sep='\n')

best_model = max(results.items(), key=lambda t: t[1][-2])
print("\nНаилучшая модель по F-мере:\n", *best_model, sep='\n')

Различные параметры классификатора KNN и оценочные метрики для каждой модели:

(normalizer, n_neighbors, metric, weight): [precision, recall, f1_score, accuracy]

('CountVectorizer', 1, 'euclidean', 'uniform'): [0.4074, 0.3873, 0.3912, 0.6000]
('CountVectorizer', 1, 'euclidean', 'distance'): [0.4074, 0.3873, 0.3912, 0.6000]
('CountVectorizer', 1, 'cosine', 'uniform'): [0.4891, 0.4668, 0.4737, 0.6466]
('CountVectorizer', 1, 'cosine', 'distance'): [0.4891, 0.4668, 0.4737, 0.6466]
('CountVectorizer', 2, 'euclidean', 'uniform'): [0.4614, 0.3840, 0.3714, 0.5997]
('CountVectorizer', 2, 'euclidean', 'distance'): [0.4018, 0.3835, 0.3847, 0.5956]
('CountVectorizer', 2, 'cosine', 'uniform'): [0.4666, 0.4507, 0.4471, 0.6481]
('CountVectorizer', 2, 'cosine', 'distance'): [0.4867, 0.4745, 0.4795, 0.6460]
('CountVectorizer', 3, 'euclidean', 'uniform'): [0.4857, 0.3661, 0.3477, 0.6679]
('CountVectorizer', 3, 'euclidean', 'distance'): [0.4344, 0.3613, 0.3420, 0.6624]
('CountVectorizer', 3, 'cosine', '

#### Обучение модели-классификатора `RandomForestClassifier` 
- с различными параметрами и разными способами векторизации слов
- оценивание каждой модели с целью выбора наилучшей

In [ ]:
def forest_cv_score(X_tr, y_train, X_te, y_test, parameters, score_functions, acc_function, forest_class):
    output = {}
    for normalizer, normalizer_name in parameters['normalizers']:
        if normalizer:
            normalizer.fit(X_tr)
            #normalizer.fit_transform(df_train.data).toarray()
            X_train = normalizer.transform(X_tr).toarray()
            X_test = normalizer.transform(X_te).toarray()

        for n_estimators in parameters['n_estimators']:
            for max_features in parameters['max_features']:
                for bootstrap in parameters['bootstrap']:
                    key = (normalizer_name, n_estimators, max_features, bootstrap)

                    classifier = forest_class(n_estimators, max_features=max_features, bootstrap=bootstrap)
                    classifier.fit(X_train, y_train)
                    y_pred = classifier.predict(X_test)

                    output[key] = [score(y_test, y_pred, average='macro', zero_division=1) 
                                    for score in score_functions]
                    output[key].append(acc_function(y_test, y_pred))
    return output

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


count_vec = CountVectorizer(analyzer=lambda x: x)
tf_idf = TfidfVectorizer(analyzer=lambda x: x)

parameters = {
    'n_estimators': [i for i in range(100, 301, 100)],
    'max_features': ['log2', 'sqrt'],
    'bootstrap': [True, False],
    'normalizers': [(count_vec, 'CountVectorizer'), (tf_idf, 'TfidfVectorizer')]
}

results = forest_cv_score(df_train['data'], df_train['class'], df_test['data'], df_test['class'], 
                          parameters, [precision_score, recall_score, f1_score], accuracy_score, 
                          RandomForestClassifier)

print("Различные параметры классификатора RandomForest и оценочные метрики для каждой модели:\n")
print("(normalizer, n_estimators, max_features, bootstrap): [precision, recall, f1_score, accuracy]\n")
print(*[f"{key}:" + " [" + ', '.join([f"{score:.4f}"for score in scores]) + "]"
          for key, scores in results.items()], sep='\n')

best_model = max(results.items(), key=lambda t: t[1][-1])
print("\nНаилучшая модель по аккуратности:\n", *best_model, sep='\n')

best_model = max(results.items(), key=lambda t: t[1][-2])
print("\nНаилучшая модель по F-мере:\n", *best_model, sep='\n')

Различные параметры классификатора RandomForest и оценочные метрики для каждой модели:

(normalizer, n_estimators, max_features, bootstrap): [precision, recall, f1_score, accuracy]

('CountVectorizer', 100, 'log2', True): [0.5800, 0.5362, 0.5531, 0.7111]
('CountVectorizer', 100, 'log2', False): [0.5646, 0.5389, 0.5497, 0.7022]
('CountVectorizer', 100, 'sqrt', True): [0.5508, 0.5222, 0.5340, 0.6884]
('CountVectorizer', 100, 'sqrt', False): [0.5255, 0.5167, 0.5208, 0.6681]
('CountVectorizer', 200, 'log2', True): [0.5734, 0.5171, 0.5365, 0.7082]
('CountVectorizer', 200, 'log2', False): [0.5687, 0.5385, 0.5510, 0.7020]
('CountVectorizer', 200, 'sqrt', True): [0.5561, 0.5286, 0.5401, 0.6926]
('CountVectorizer', 200, 'sqrt', False): [0.5246, 0.5169, 0.5204, 0.6668]
('CountVectorizer', 300, 'log2', True): [0.5809, 0.5231, 0.5434, 0.7098]
('CountVectorizer', 300, 'log2', False): [0.5696, 0.5346, 0.5486, 0.7048]
('CountVectorizer', 300, 'sqrt', True): [0.5544, 0.5262, 0.5378, 0.6915]
('CountVec